In [23]:
#import dependencies
import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

In [24]:
#reading in the cities.csv created from weatherpy
path = '../weatherpy/outputs/cities.csv'
cities_for_heatmap_df = pd.read_csv(path)
cities_for_heatmap_df.sample(10)

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement
331,victor harbor,-35.57,138.62,61.00,1,75,1.99,AU,1608823827
284,aksarka,66.56,67.80,-39.12,25,81,5.21,RU,1608823822
422,kindu,-2.95,25.95,85.51,6,60,1.99,CD,1608823749
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842
526,tromso,69.65,18.96,33.80,0,74,18.34,NO,1608823793
291,marquette,46.53,-87.63,14.00,100,71,1.01,US,1608823823
207,saint-francois,46.42,3.91,46.40,100,87,10.29,FR,1608823797
518,bodden town,19.28,-81.25,84.99,20,74,9.17,KY,1608823844
255,belmonte,-15.86,-38.88,86.00,20,66,5.82,BR,1608823801
71,hasaki,35.73,140.83,53.60,40,66,8.05,JP,1608823790


In [25]:
#checking max humidity in cities_for_heatmap_df
cities_for_heatmap_df['humidity'].max()

100

In [26]:
#creating weighted (by humidity) heatmap
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
#creating a new df from filtered cities_for_heatmap_df
ideal_weather_df = pd.DataFrame(cities_for_heatmap_df[(cities_for_heatmap_df['max_temp'] < 70) & (cities_for_heatmap_df['max_temp'] > 60) & (cities_for_heatmap_df['wind_speed'] < 10) & (cities_for_heatmap_df['cloudiness'] == 0)])
#adding a new column for hotel names
ideal_weather_df['hotel_name'] = " "
ideal_weather_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,


In [36]:
#creating a copy of ideal_weather_df to use for google maps coordinates
true_ideal_weather_df = ideal_weather_df.copy()

In [29]:
#using ideal_weather_df and google api nearby search to find hotels near the cities
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&type={target_type}&key={g_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}')
        hotel_names.append('n/a')
ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

No hotel within 5000 m of tiznit


,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Bredasdorp Country Manor
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,n/a
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Htein Htein Thar 2
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Lake Forbes Motel
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,ديوانية شيخ عبود الذياب آل سالم
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,Pathein Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,บ้านวรฉัตร หล่มสัก
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,منزل الصابر أبوفانة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Mubashar Quarters


In [39]:
#testing to see if error is because of lack of hotels w/in 5000 m; increasing to 10000 m, then 20000 m
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&type={target_type}&key={g_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, will try 10000 m range')
        try:
            target_radius = 10000
            target_location = f'{row[1]},{row[2]}'
            final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={g_key}'
            data = requests.get(final_url).json()
            hotel_names.append(f"{data['results'][0]['name']}* 10 km away")
            target_radius = 5000
        except IndexError:
            print(f'No hotel within 10000 m of {row[0]}, will try 20000 m range')
            try:
                target_radius = 20000
                target_location = f'{row[1]},{row[2]}'
                final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={g_key}'
                data = requests.get(final_url).json()
                hotel_names.append(f"{data['results'][0]['name']}* 20 km away")
                target_radius = 5000
            except IndexError:
                print(f'no hotel found within 20000 m of {row[0]}, exiting row')
                continue

ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

No hotel within 5000 m of tiznit, will try 10000 m range
No hotel within 10000 m of tiznit, will try 20000 m range


,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Bredasdorp Country Manor
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,Hotel kerdous* 20 km away
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Htein Htein Thar 2
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Lake Forbes Motel
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,ديوانية شيخ عبود الذياب آل سالم
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,Pathein Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,บ้านวรฉัตร หล่มสัก
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,منزل الصابر أبوفانة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Mubashar Quarters


In [37]:
#using google maps coordinates of the cities to reflect city locations better
true_lats = []
true_lngs = []
base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
key=g_key
for index, row in ideal_weather_df.iterrows():
    try:
        input_query = f'{row[0]}, {row[7]}'
        final_url = f'{base_url}&input={input_query}&inputtype=textquery&fields=name,geometry&key={g_key}'
        data = requests.get(final_url).json()
        true_lats.append(data['candidates'][0]['geometry']['location']['lat'])
        true_lngs.append(data['candidates'][0]['geometry']['location']['lng'])
    except KeyError:
        print(f'Error with {row[0]}')
true_ideal_weather_df['lat'] = true_lats
true_ideal_weather_df['long'] = true_lngs
true_ideal_weather_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.538522,20.056877,69.80,0,56,8.05,ZA,1608823786,
102,tiznit,29.693392,-9.732157,66.16,0,27,6.51,MA,1608823791,
368,magway,20.154431,94.945480,67.59,0,62,3.40,MM,1608823831,
426,forbes,-33.379879,148.006689,64.99,0,52,7.23,AU,1608823836,
480,shadegan,30.654597,48.653927,60.80,0,72,9.17,IR,1608823841,
487,pathein,16.775361,94.738101,67.95,0,79,5.26,MM,1608823823,
493,lom sak,16.777016,101.242386,68.95,0,76,3.80,TH,1608823842,
548,bani walid,31.797583,14.053676,62.22,0,50,5.14,LY,1608823847,
551,turbat,26.008055,63.038306,63.84,0,30,4.43,PK,1608823847,


In [38]:
#using nearby search and true_ideal_weather_df to find hotels
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_type = 'lodging'
for index, row in true_ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&type={target_type}&key={g_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}')
        hotel_names.append('n/a')
true_ideal_weather_df['hotel_name'] = hotel_names
true_ideal_weather_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.538522,20.056877,69.80,0,56,8.05,ZA,1608823786,Bredasdorp Country Manor
102,tiznit,29.693392,-9.732157,66.16,0,27,6.51,MA,1608823791,Hotel Café Restaurant Assaka
368,magway,20.154431,94.945480,67.59,0,62,3.40,MM,1608823831,Htein Htein Thar 2
426,forbes,-33.379879,148.006689,64.99,0,52,7.23,AU,1608823836,Lake Forbes Motel
480,shadegan,30.654597,48.653927,60.80,0,72,9.17,IR,1608823841,najeh.saleh
487,pathein,16.775361,94.738101,67.95,0,79,5.26,MM,1608823823,Pathein Hotel
493,lom sak,16.777016,101.242386,68.95,0,76,3.80,TH,1608823842,บ้านวรฉัตร หล่มสัก
548,bani walid,31.797583,14.053676,62.22,0,50,5.14,LY,1608823847,قصر سيدي سليم
551,turbat,26.008055,63.038306,63.84,0,30,4.43,PK,1608823847,BOQ's


In [33]:
#creating symbol layer atop weighted heatmap based on true_ideal_weather_df
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in true_ideal_weather_df.iterrows()]
hotel_locations = true_ideal_weather_df[["lat", "long"]]


marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))